# Azure SQL Database
It is a cloud-based SQL Database that stores structured data.
<br/>For more information visit:
<br/>https://docs.microsoft.com/en-us/azure/azure-sql/database/sql-database-paas-overview
<br/>In this example we will be commiting structed sensor data in a CSV to a SQL database

### Installing packages

In [15]:
import pyodbc
import pandas as pd

### Create dataframe object
The dataframe will contaoin the data from the CSV file that is to be commited to the sql database

In [18]:
#Read CSV file to pandas dataframe
df = pd.read_csv("sittingone_MetaOn_Gyr.csv") 

#Display the top five rows
df.head()

,epoch (ms),time (06:00),elapsed (s),x-axis (deg/s),y-axis (deg/s),z-axis (deg/s)
0,1600875978480,2020-09-23T18:46:18.480,0.00,-13.476,0.488,0.915
1,1600875978500,2020-09-23T18:46:18.500,0.02,-14.756,0.427,0.427
2,1600875978520,2020-09-23T18:46:18.520,0.04,-18.171,2.012,-0.122
3,1600875978540,2020-09-23T18:46:18.540,0.06,-12.195,1.524,-0.183
4,1600875978560,2020-09-23T18:46:18.560,0.08,-2.683,-0.610,0.122


In [19]:
#Renaming the columns for easier reference(optional)
df.rename(columns = {'x-axis (deg/s)': 'x', 'y-axis (deg/s)': 'y', 'z-axis (deg/s)': 'z'}, inplace = True) 

df.head()

,epoch (ms),time (06:00),elapsed (s),x,y,z
0,1600875978480,2020-09-23T18:46:18.480,0.00,-13.476,0.488,0.915
1,1600875978500,2020-09-23T18:46:18.500,0.02,-14.756,0.427,0.427
2,1600875978520,2020-09-23T18:46:18.520,0.04,-18.171,2.012,-0.122
3,1600875978540,2020-09-23T18:46:18.540,0.06,-12.195,1.524,-0.183
4,1600875978560,2020-09-23T18:46:18.560,0.08,-2.683,-0.610,0.122


### Connect to Database
You can obtain the connection details from [Azure Portal](https://docs.microsoft.com/en-us/azure/storage/common/storage-account-create?tabs=azure-portal), [Azure PowerShell](https://docs.microsoft.com/en-us/azure/storage/common/storage-account-create?tabs=azure-powershell) or [Azure CLI](https://docs.microsoft.com/en-us/azure/storage/common/storage-account-create?tabs=azure-cli)

In [ ]:
#Create connection string

server = 'server-name.database.windows.net'
database = 'DBName'
username = 'user'
password = 'yourpassword'
DATABASE_URI = 'DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password

In [ ]:
#Connect to SQL Server
conn = pyodbc.connect(DATABASE_URI)
cursor = conn.cursor()

### Create a table 
Create a table that will contain the structured data 

In [ ]:
# Create a Table if one with same name doesn't already exist
cursor.execute('''
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES
           WHERE TABLE_NAME = N'your_table_name')
BEGIN
     CREATE TABLE your_table_name (Epoch bigint, X_axis float, Y_axis float, Z_axis float)
END '''
)
print('Table successfully created!')

###  Commit database
The data will be uoloaded to Azure SQL database row by row. This may take a considerable amount of time depending on the number of rows 

In [ ]:
# Insert DataFrame to Table
for row in acc_df.itertuples():
    cursor.execute('''
                    INSERT INTO DBName.dbo.body_acc_all (Epoch, X_axis, Y_axis, Z_axis)
                    VALUES (?,?,?,?,?)
                    ''',
                    row.Epoch, 
                    row.X_axis,
                    row.Y_axis,
                    row.Z_axis,
                    )


#Commit Database
conn.commit()
print("Finished!")

#Close connection
cursor.close()